# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts learned in this section, from adding interactions and polynomials to your model to AIC and BIC!

## Summary

You will be able to:
- Build a linear regression model with polynomial features/interactions
- Perform regularization
- Use AIC and BIC to select the best value for the regularization parameter


## Let's get started!

Import all the necessary packages.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import preprocessing

from sklearn.datasets import load_boston

## Look at a Baseline Boston Housing Data Model

Import the Boston housing data set, use all the predictors in their scaled version (using `preprocessing.scale`. Look at a baseline model using *scaled variables* as predictors. Use 5-fold cross-validation this time and use the $R^2$ score to evaluate the model.

In [6]:
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.DataFrame(boston.target, columns=["target"])

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
scaler.fit(X) 
scaled_X = scaler.transform(X) 

In [3]:


from sklearn import linear_model
model = linear_model.LinearRegression()

from sklearn.model_selection import cross_val_score, cross_val_predict
scores = cross_val_score(model, scaled_X, y, cv=5)
predictions = cross_val_predict(model, scaled_X, y, cv=5)

In [4]:
import numpy as np

In [5]:
np.mean(scores)

0.3532759243958826

## Include interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Next, evaluate that model using 5-fold classification and store the $R^2$ to compare it with the baseline model.

You've created code for this before in the interactions lab, yet this time, you have scaled the variables so the outcomes may look different. 

Print the 7 most important interactions.

In [26]:
from itertools import combinations
combinations = list(combinations(X.columns, 2))

## code to find top 7 interactions by R^2 value here
scores = []
for comb in list(combinations):
    X_interact = X.copy()
    X_interact["interaction"] = X[comb[0]] * X[comb[1]]
    crossvalidation = KFold(n_splits=3, shuffle=True, random_state=1)
    interaction_model_score = np.mean(cross_val_score(model, 
                                        X_interact, 
                                        y, 
                                        scoring="r2", 
                                        cv=crossvalidation))
    scores.append((comb, interaction_model_score))
print(sorted(scores, key=lambda x: x[1])[-7:])

[(('RM', 'AGE'), 0.7423308796795774), (('RM', 'B'), 0.743125275491411), (('INDUS', 'RM'), 0.7566142675433983), (('RM', 'PTRATIO'), 0.7600538840370085), (('RM', 'RAD'), 0.767091186595275), (('RM', 'TAX'), 0.774093888685694), (('RM', 'LSTAT'), 0.7846518153987837)]


Write code to include the 7 most important interactions in your data set by adding 7 columns. Name the columns "var1_var2" with var1 and var2 the two variables in the interaction.

In [28]:
X_interact = X.copy()
for comb in [x[0] for x in sorted(scores, key=lambda x: x[1])[-7:]]:   
    X_interact[f"{comb[0]}_{comb[1]}"] = X[comb[0]] * X[comb[1]]
crossvalidation = KFold(n_splits=5, shuffle=True, random_state=1)
interaction_model_score = np.mean(cross_val_score(model, 
                                        X_interact, 
                                        y, 
                                        scoring="r2", 
                                        cv=crossvalidation))
interaction_model_score

0.7875386050321879

## Include Polynomials

Try polynomials of 2, 3 and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of 4, the particular column is raised to the power of 2 and 3 as well in other terms. We only want to include "pure" polynomials, so make sure no interactions are included. We want the result to return a list that contain tuples of the form:

`(var_name, degree, R2)`, so eg. `('DIS', 3, 0.732)`

In [12]:
scores = {}
for feature in list(X.columns):
    scores[feature] = []
    for power in range(2,5):
        X_interact = X.copy()
        X_interact["interaction"] = X[feature]**power
        crossvalidation = KFold(n_splits=3, shuffle=True, random_state=1)
        interaction_model_score = np.mean(cross_val_score(model, 
                                            X_interact, 
                                            y, 
                                            scoring="r2", 
                                            cv=crossvalidation))
        scores[feature].append((feature, power, interaction_model_score))

For each variable, print out the maximum R2 possible when including Polynomials.

In [15]:
for feature in scores.keys():
    print(sorted(scores[feature], key=lambda x: x[2])[-1])

('CRIM', 4, 0.7179210367372043)
('ZN', 4, 0.7237962553166085)
('INDUS', 2, 0.7245599863495578)
('CHAS', 4, 0.7190936670978688)
('NOX', 2, 0.7192755557369853)
('RM', 2, 0.7751449457900822)
('AGE', 4, 0.7228580251727251)
('DIS', 2, 0.7351176726368157)
('RAD', 2, 0.7184030577078094)
('TAX', 2, 0.7204619469261772)
('PTRATIO', 3, 0.72221070878329)
('B', 4, 0.7234582043897669)
('LSTAT', 2, 0.7705741444055368)


Which two variables seem to benefit most from adding Polynomial terms?

RM and LSTAT

Add Polynomials for the two features that seem to benefit the most, as in have the best R squared compared to the baseline model. For each of the two feature, raise to the Polynomial that generates the best result. Make sure to start from the data set `df_inter` so the final data set has both interactions and polynomials in the model.

In [29]:
X_interact["RM_2"] = X["RM"]**2
X_interact["LSTAT_2"] = X["LSTAT"]**2

In [30]:
interaction_model_score = np.mean(cross_val_score(model, 
                                            X_interact, 
                                            y, 
                                            scoring="r2", 
                                            cv=crossvalidation))
print(interaction_model_score)

0.7941194678745938


check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [31]:
X_interact.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'RM_AGE', 'RM_B', 'INDUS_RM', 'RM_PTRATIO',
       'RM_RAD', 'RM_TAX', 'RM_LSTAT', 'RM_2', 'LSTAT_2'],
      dtype='object')

## Full model R-squared

Check out the R-squared of the full model.

In [32]:
print(interaction_model_score)

0.7941194678745938


## Finding the best Lasso regularization parameter

You've learned that, when using Lasso regularization, your coefficients shrink to 0 when using a higher regularization parameter. Now the question is which value we should choose for the regularization parameter. 

This is where the AIC and BIC come in handy! We'll use both criteria in what follows and perform cross-validation to select an optimal value of the regularization parameter alpha of the Lasso estimator.

Read the page here: https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html and create a similar plot as the first one listed on the page. 

In [38]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [40]:
y.head()

,target
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


In [8]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC

# This is to avoid division by zero while doing np.log10
EPSILON = 1e-4

# #############################################################################
# LassoLarsIC: least angle regression with BIC/AIC criterion

model_bic = LassoLarsIC(criterion='bic')
model_bic.fit(scaled_X, y)
alpha_bic_ = model_bic.alpha_

model_aic = LassoLarsIC(criterion='aic')
model_aic.fit(X_scaled, y)
alpha_aic_ = model_aic.alpha_


def plot_ic_criterion(model, name, color):
    alpha_ = model.alpha_ + EPSILON
    alphas_ = model.alphas_ + EPSILON
    criterion_ = model.criterion_
    plt.plot(-np.log10(alphas_), criterion_, '--', color=color,
             linewidth=3, label='%s criterion' % name)
    plt.axvline(-np.log10(alpha_), color=color, linewidth=3,
                label='alpha: %s estimate' % name)
    plt.xlabel('-log(alpha)')
    plt.ylabel('criterion')

plt.figure()
plot_ic_criterion(model_aic, 'AIC', 'b')
plot_ic_criterion(model_bic, 'BIC', 'r')
plt.legend()
plt.title('Information-criterion for model selection')

# #############################################################################
# LassoCV: coordinate descent

# Compute paths
print("Computing regularization path using the coordinate descent lasso...")
model = LassoCV(cv=5).fit(X, y)

# Display results
m_log_alphas = -np.log10(model.alphas_ + EPSILON)

plt.figure()
#ymin, ymax = 2300, 3800
plt.plot(m_log_alphas, model.mse_path_, ':')
plt.plot(m_log_alphas, model.mse_path_.mean(axis=-1), 'k',
         label='Average across the folds', linewidth=2)
plt.axvline(-np.log10(model.alpha_ + EPSILON), linestyle='--', color='k',
            label='alpha: CV estimate')

plt.legend()

plt.xlabel('-log(alpha)')
plt.ylabel('Mean square error')
plt.title('Mean square error on each fold: coordinate descent ')
plt.axis('tight')
#plt.ylim(ymin, ymax)

# #############################################################################
# LassoLarsCV: least angle regression

# Compute paths
print("Computing regularization path using the Lars lasso...")
model = LassoLarsCV(cv=5).fit(X, y)

# Display results
m_log_alphas = -np.log10(model.cv_alphas_ + EPSILON)

plt.figure()
plt.plot(m_log_alphas, model.mse_path_, ':')
plt.plot(m_log_alphas, model.mse_path_.mean(axis=-1), 'k',
         label='Average across the folds', linewidth=2)
plt.axvline(-np.log10(model.alpha_), linestyle='--', color='k',
            label='alpha CV')
plt.legend()

plt.xlabel('-log(alpha)')
plt.ylabel('Mean square error')
plt.title('Mean square error on each fold: Lars')
plt.axis('tight')
#plt.ylim(ymin, ymax)

plt.show()

NameError: name 'X_scaled' is not defined

## Analyze the final result

Finally, use the best value for regularization parameter according to AIC and BIC and compare the R squared parameters and MSE using train-test-split. Compare with the baseline model.

In [44]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

reg = LinearRegression().fit(X_train, y_train)

print(reg.score(X_test, y_test))
print(reg.coef_)
print(reg.intercept_)

0.7261570836552483
[[-1.28749718e-01  3.78232228e-02  5.82109233e-02  3.23866812e+00
  -1.61698120e+01  3.90205116e+00 -1.28507825e-02 -1.42222430e+00
   2.34853915e-01 -8.21331947e-03 -9.28722459e-01  1.17695921e-02
  -5.47566338e-01]]
[33.33497576]


In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

from sklearn import linear_model
reg = linear_model.Lasso(alpha=np.exp(-3))
reg.fit(X_train, y_train)

print(reg.score(X_test, y_test))
print(reg.coef_)
print(reg.intercept_)

0.7091450658575456
[-0.11562299  0.0398009  -0.00481961  2.35312461 -0.          3.86151788
 -0.02465818 -1.15775668  0.20121511 -0.00988358 -0.74721749  0.0123206
 -0.59362496]
[23.17227906]


In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

from sklearn import linear_model
reg = linear_model.Lasso(alpha=np.exp(-1.5))
reg.fit(X_train, y_train)

print(reg.score(X_test, y_test))
print(reg.coef_)
print(reg.intercept_)

0.7067119613006001
[-0.11576423  0.04025736  0.          0.         -0.          3.39795051
 -0.01566924 -1.09235194  0.21916877 -0.01045975 -0.76013977  0.01264704
 -0.64126055]
[26.14073059]


## Level Up - Optional

### Create a Lasso Path

From this section, you know that when using lasso, more parameters shrink to zero as your regularization parameter goes up. In Scikit-Learn there is a function lasso_path which visualizes the shrinkage of the coefficients while alpha changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for subset selection
This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Boston Housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to create better linear models and how to use AIC and BIC for both feature selection and to optimize your regularization parameter when performing Ridge and Lasso. 